# About: VCノードに関するパラメータの設定

---

VCノードに関するパラメータ設定を行います

# 構成

![moodleの構成](image/moodle-1-02.png)

この一連のNotebookではDockerコンテナを用いて[Moodle](https://moodle.org/)環境の構築を行います。

構築するMoodle環境は次の３つのコンテナで構成されています。

* Moodle
  - MoodleのPHPスクリプトを実行するコンテナ
* [MySQL](https://hub.docker.com/_/mysql/)
  - Moodleの設定を保存するデータベース
* [Nginx](https://hub.docker.com/_/nginx/)
  - Moodleサーバに対するリバースプロキシ

ここでは小規模構成を想定して３つのコンテナを１つのVCノードに配置する構成とします。
またMoodleとMySQLコンテナに対しては、データを保存するためのEBSをそれぞれのコンテナに割り当てます。

# Ansibleの設定

Notebook環境からVCノードの環境構築を行うために構成管理ツールの[Ansible](https://www.ansible.com/)を利用します。ここではAnsibleに関する設定を行います。
![Ansibleの設定](image/moodle-1-03.png)

## インベントリのパスを設定する

Ansibleでは操作対象となるホストのリストをインベントリとして管理しています。ここではそのインベントリとして扱うファイルのパスを設定します。

インベントリのパスは[デフォルト設定](http://docs.ansible.com/ansible/latest/reference_appendices/config.html#default-host-list)では /etc/ansible/hosts になっています。これを`/notebooks/notebook/handson/moodle/hosts`に変更します。Ansible の設定値は`./ansible.cfg` にパラメータを書き込むことで変更できます。

次のセルを実行することで `ansible.cfg` に必要なパラメータを設定できます。 

In [1]:
import os

with open('./ansible.cfg', 'w') as f:
    f.write('''[defaults]
inventory = {}
'''.format(os.path.join(os.getcwd(), 'hosts')))

# 作成した ansible.cfg を表示
!cat ansible.cfg

[defaults]
inventory = /notebooks/notebook/handson/moodle/hosts


## グループ名を指定する

Ansible の操作対象となるグループ名を指定します。

Ansibleからリモートホストを操作するには対象となるホスト名（IPアドレス）を指定する必要があります。VCノードのIPアドレスは確保する毎に異なる値となる可能性があるので操作対象の指定に利用するのは適切ではありません。Ansible では複数のホストをまとめてグループとして扱うことができるので、操作対象の指定にはホスト名ではなくグループ名を利用することにします。

In [2]:
target_group = 'Moodle'  # 操作対象のグループ名

### チェックと保存

設定されたグループ名に対応する group_vars ファイルを作成してグループ名を記録します。
既に同名の group_vars ファイルが存在している場合は、他の Notebook で利用中のグループ名であると判定してエラーとなります。

In [3]:
import os
group_vars_file = os.path.join(os.getcwd(), 'group_vars', target_group + '.yml')
if os.path.exists(group_vars_file):
    raise RuntimeError("ERROR: already exists: {}".format(group_vars_file))

# グループ名を保存する
%run scripts/group.py
update_group_vars(target_group, target_group=target_group)

!cat group_vars/{target_group}.yml

target_group: Moodle


## ログインユーザ名を指定する

VCノードにログインする際のユーザ名を指定してください。

> ここで指定したユーザがVCノードの起動時に自動的に作成されます。

In [4]:
ansible_user = 'ansible'

### チェックと保存

In [5]:
import re
%run scripts/group.py

# 簡単なユーザ名のチェック
username_pattern = re.compile(r'[a-zA-Z0-9_.][a-zA-Z0-9_.-]{0,30}[a-zA-Z0-9_.$-]?$')
if not username_pattern.match(ansible_user):
    raise RuntimeError("ERROR: invalid user name '{}'".format(ansible_user))
    
# ユーザ名の保存
update_group_vars(target_group, ansible_user=ansible_user)

## ssh公開鍵を指定する

VCノードにログインする際に利用するSSHの公開鍵ファイルの設定を行います。

> ハンズオン環境では事前に公開鍵(~/.ssh/id_rsa_moodle)を作成してあります。

In [6]:
import os

# 秘密鍵ファイルを設定する
ansible_ssh_private_key_file = os.path.expanduser('~/.ssh/id_rsa_moodle')

# 公開鍵ファイルを設定する。
ansible_ssh_public_key_file = ansible_ssh_private_key_file + '.pub'

### チェックと保存

sshの鍵ファイルが存在していることを確認します。存在していたら group_vars ファイルに設定されたパスを保存します。

In [7]:
# ファイルの存在チェック
for path in [ansible_ssh_private_key_file, ansible_ssh_public_key_file]:
    if not os.path.exists(path):
        raise RuntimeError("ERROR: not exists: {}".format(path))

# 保存
update_group_vars(
    target_group,
    ansible_ssh_private_key_file=ansible_ssh_private_key_file,
    ansible_ssh_public_key_file=ansible_ssh_public_key_file,
)

## sshオプションを指定する

VCノードにSSHで接続する際のsshのオプションを設定します。

ここでは新規ホストを追加した際の操作の煩雑さを避けるため、以下のオプションを設定します。
* UserNownHostsFile=/dev/null (known_hostsに記録しないようにする)
* StrictHostKeyChecking=no (ユーザに問い合わせることなく無条件にknown_hostsにホストキーを追加する)

In [8]:
ansible_ssh_extra_args = '-o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null'

### 保存


In [9]:
update_group_vars(
    target_group,
    ansible_ssh_extra_args=ansible_ssh_extra_args
)

## 保存したパラメータの確認

ここまで group_vars ファイルに保存したパラメータの内容を確認します。

In [10]:
!cat group_vars/{target_group}.yml

ansible_ssh_extra_args: -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null
ansible_ssh_private_key_file: /home/bit_kun/.ssh/id_rsa_moodle
ansible_ssh_public_key_file: /home/bit_kun/.ssh/id_rsa_moodle.pub
ansible_user: ansible
target_group: Moodle


# VCP Systemとの接続確認

![VCP Systemとの接続確認](image/moodle-1-04.png)

VCP Systemに接続できることを確認します。

> ハンズオン環境ではVCP Systemに接続するためのアクセストークンは事前に設定ファイルに書き込まれています。

## VCコントローラとの接続を確認する

VCP SystemのVC コントローラに接続できることを確認します。

In [11]:
from vcpsdk import vcpsdk
from datetime import datetime
%run scripts/group.py

vcc_access_token = load_group_var(target_group, 'vcc_access_token')

# 指定されたアクセストークンでVCコントローラに接続できるかチェックする
sdk = vcpsdk.VcpSDK(
    vcc_access_token, 
    'vcksdk_check_{:%Y%m%d%H%M%S}'.format(datetime.now()))
print(sdk)

# VCP SDK バージョン確認
sdk.version()

# チェックのためのVCP SDK を削除する
sdk.cleanup(wait_for=True)

vc_name: vcksdk_check_20180621155618

[Vc]
+ name[vcksdk_check_20180621155618] vcno(22) state[RUNNING] vcid[1a2589f155264139b5f9359018e7f927]




vcplib:
  filename: /notebooks/notebook/vcplib/occtr.py
  version: 18.04.1+201805010

vcpsdk:
  filename: /notebooks/notebook/vcpsdk/vcpsdk.py
  version: 18.04.1+20180510

  plugin:
    aws: 1.0+20180331
    azure: 1.0+20180331
    sakura: 1.0+20180331
    aic: 1.0+20180331
    gcp: 1.0+20180331
    onpremises: 1.0+20180331

server:
    host: 192.168.2.1
    name: VCC511


## HashiCorp Vaultとの接続を確認する

VCPでは秘密情報を管理するために HashiCorp Vault サーバを用意しています（以降ではVaultサーバと記します）。このNotebookではMoodleの管理者パスワード、データベースのユーザパスワード、EBSの暗号化鍵ファイルなどの管理に利用しています。

Vaultサーバに接続できることを確認します。

In [12]:
%run scripts/group.py
# Vaultサーバのアドレスを読み込む
vault_address = load_group_var(target_group, 'vault_address')
print(vault_address)

# Vaultサーバのアクセストークンを読み込む
vault_token = load_group_var(target_group, 'vcc_access_token')

# Vaultサーバへのアクセス
!curl --fail --header "X-Vault-Token:{vault_token}" \
    "{vault_address}/v1/cubbyhole?help=1"

http://192.168.2.1:8200
{"help":"## DESCRIPTION\n\nThe cubbyhole backend reads and writes arbitrary secrets to the backend.\nThe secrets are encrypted/decrypted by Vault: they are never stored\nunencrypted in the backend and the backend never has an opportunity to\nsee the unencrypted value.\n\nThis backend differs from the 'generic' backend in that it is namespaced\nper-token. Tokens can only read and write their own values, with no\nsharing possible (per-token cubbyholes). This can be useful for implementing\ncertain authentication workflows, as well as \"scratch\" areas for individual\nclients. When the token is revoked, the entire set of stored values for that\ntoken is also removed.\n\n## PATHS\n\nThe following paths are supported by this backend. To view help for\nany of the paths below, use the help command with any route matching\nthe path pattern. Note that depending on the policy of your auth token,\nyou may or may not be able to access certain paths.\n\n    ^.*$\n        Pas

Vaultサーバのアクセスに失敗した場合はエラーとなります。エラーとなった場合はcurlコマンドのエラー出力を確認して適切な対応を行ってください。

よくあるエラー出力とその対応方法について以下に記します。

* エラー出力: ```curl: (7) Failed to connect to 172.30.XXX.XXX port 8200: Connection refused```
  - 原因
    - 設定されているVaultサーバのアドレスが正しくありません
  - 対応
    - Vaultサーバのアドレスを設定しなおしてください

* エラー出力: ```curl: (22) The requested URL returned error: 403 Forbidden```
  - 原因
    - Vaultサーバのアクセストークンが正しくありません
  - 対応
    - Vaultサーバのアクセストークンを設定しなおしてください
    - Vaultサーバが正しく設定されているかVCP管理者に問い合わせてください
    

# VCノードに関する設定

![VCノードに関する設定](image/moodle-1-05.png)

## クラウドプロバイダを選択する

VCノードを起動するプロバイダを指定します。

VCPがシステムとして指定できるプロバイダの値を以下に示します。

* aws
* azure
* gcp
* sakura

ただし、実際に上記の全てのプロバイダが利用できるわけではなく、実行環境毎に利用できるプロバイダは異なります。

ハンズオン環境では `aws` のみが利用できます。

In [13]:
# (例)
# vcp_provider = 'aws'
# vcp_provider = 'azure'

vcp_provider = 'aws'

### チェックと保存

設定されたプロバイダ名が妥当かどうかチェックする。

> ここでは形式的なチェックのみ行い、実際にこの環境から指定されたプロバイダが利用できるかどうかまでは確認しない。

設定されたプロバイダ名が妥当である場合は、その値を group_vars ファイルに保存する。

In [14]:
# プロバイダ名の妥当性チェック
#if vcp_provider not in ['aws', 'azure', 'gcp', 'sakura']:
if vcp_provider not in ['aws']:
    raise RuntimeError("ERROR: invalid name: {}".format(vcp_provider))

# 保存
update_group_vars(
    target_group,
    vcp_provider=vcp_provider,
)

## 割り当てるリソース量を指定する

VCノードに割り当てるリソース量を指定します。

リソース量の指定方法は各クラウドプロバイダ毎に異なります。VCP SDKではそれらを抽象化した `flavor` でリソースの割り当て量を指定することができます。ここではVCノードに割り当てるリソース量を `flavor` で指定します。

In [15]:
# (例)
# vcp_flavor = 'small'

vcp_flavor = 'small'

### チェックと保存

設定された値の妥当性をチェックして問題なければ group_vars ファイルに保存する。

In [16]:
from vcpsdk.vcp_config_yaml import VcpConfigYaml
%run scripts/group.py

vcp_provider = load_group_var(target_group, 'vcp_provider')
flavor_config = VcpConfigYaml().load('vcp_flavor')
if vcp_flavor not in flavor_config[vcp_provider]:
    raise RuntimeError("ERROR: invalid value: {}".format(vcp_flavor))

update_group_vars(
    target_group,
    vcp_flavor=vcp_flavor,
)

## IPアドレスを確認する

VCP SDKではVCノードの起動時に適切なプライベートIPアドレスを自動的に割り当てます。また、自動的な割り当てを行わずに事前に設定したIPアドレスをVCノードに割り当てることもできます。MoodleなどのサーバをNATインスタンスを経由してインターネットに公開する場合は、NATインスタンスのDNAT設定とアドレスを合わせるために事前にIPアドレスを設定するほうが適しています。

ハンズオン環境ではNATインスタンスの設定に合わせたIPアドレスの設定を事前に行っています。設定されているIPアドレスの値を確認します。

In [17]:
%run scripts/group.py
vc_node_address = load_group_var(target_group, 'vc_node_address')
print(vc_node_address)

172.30.2.100


## 保存したパラメータの確認

ここまで group_vars ファイルに保存したパラメータの内容を確認します。

In [18]:
!cat group_vars/{target_group}.yml

ansible_ssh_extra_args: -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null
ansible_ssh_private_key_file: /home/bit_kun/.ssh/id_rsa_moodle
ansible_ssh_public_key_file: /home/bit_kun/.ssh/id_rsa_moodle.pub
ansible_user: ansible
target_group: Moodle
vcp_flavor: small
vcp_provider: aws


# ボリューム(EBS)に関する設定

![ボリュームに関する設定](image/moodle-1-06.png)



このNotebookが構築するMoodle環境では３つのEBSを使用しています。

* ルートボリューム
   - VCノードのルートボリューム
   - OSなどがインストールされている
* Moodle用ボリューム
   - Moodle のデータとPHPファイルを格納する
* MySQL用ボリューム
   - MySQL のデータを格納する
   
それぞれのボリュームに対して以下のパラメータを設定します。

* サイズ(GB)
* ボリュームタイプ
* マウントポイント
* ファイルシステム

> アプリケーションテンプレートのNotebookではMoodle/MySQL用のEBSをNotebookから作成しているため、上記4つのパラメータを指定する必要があります。今回のハンズオンでは事前にMoodle/MySQL用のEBSを作成しているため、マウントポイントとファイルシステムの指定のみが必要となります。サイズとボリュームタイプに関しては事前に設定した値の確認のみ行います。

## ルートボリュームサイズを指定する

ルートボリュームについてはサイズ(GB)のみを指定することができます。ただしルートボリュームサイズは、 通常「[5.2 割り当てるリソース量を指定する](#割り当てるリソース量を指定する)」で設定した `flavor` により適切な値が選択されます。`flavor`で設定される値と**異なるサイズを指定したい**場合のみ、ここで値を設定してください。

In [19]:
# (例)
# root_volume_size = 32

### 保存

In [20]:
if 'root_volume_size' in vars():
    %run scripts/group.py
    update_group_vars(
        target_group,
        root_volume_size=root_volume_size,
    )

## Moodle用ボリュームのパラメータ設定

Moodle用ボリュームに関するパラメータを指定してください。

### Moodle用ボリュームのサイズを確認する

Moodle用ボリュームのサイズ(GB)を指定します。


> ハンズオン環境では事前に設定しているので設定値の確認のみ行います。

In [21]:
# (例)
# volume_moodle_size = 320

print(load_group_var(target_group, 'volume_moodle_size'))

8


### Moodle用ボリュームのボリュームタイプを確認する

ボリュームタイプを指定します。

> ハンズオン環境では事前に設定しているので設定値の確認のみ行います。

指定できるタイプはクラウドプロバイダ毎に異なります。どのような値が指定できるかについては以下のリンクを参考にしてください。

* AWS
  - [Amazon EBS ボリュームの種類](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/EBSVolumeTypes.html)
* Azure
  - [Managed Disks の価格](https://azure.microsoft.com/ja-jp/pricing/details/managed-disks/)

In [22]:
# (例)
# AWSの場合
# volume_moodle_type = 'gp2'
# Azureの場合
# volume_moodle_type = 'Standard_LRS'

print(load_group_var(target_group, 'volume_moodle_type'))

gp2


### Moodle用ボリュームのマウントポイントを指定する

Moodle用ボリュームをマウントするディレクトリを指定してください。

In [23]:
volume_moodle_mount = '/opt/moodle'

#### 保存

In [24]:
%run scripts/group.py
update_group_vars(
    target_group,
    volume_moodle_mount=volume_moodle_mount,
)

### Moodle用ボリュームのファイルシステムを指定する


Moodle用ボリュームのファイルシステムを指定してください。

今のところ指定できる値は **btrfs のみ**になっています。

In [25]:
volume_moodle_fs = 'btrfs'

ファイルシステムをマウントする際のオプションを指定してください。ここで値が指定されない場合はデフォルトの設定でマウントされます。

In [26]:
# (例)
# volume_moodle_mount_opt = 'noatime,autodefrag,compress-force=lzo,space_cache'

#### 保存

In [27]:
# サポートしているファイルシステムが指定されているかをチェックする
support_fs = ['btrfs']
if volume_moodle_fs not in support_fs:
    raise RuntimeError(
        "ERROR: Unsupported file system: {}".format(volume_moodle_fs))

update_group_vars(
    target_group,
    volume_moodle_fs=volume_moodle_fs,
)

update_group_vars(
    target_group,
    volume_moodle_mount_opt=(
        volume_moodle_mount_opt
        if 'volume_moodle_mount_opt' in vars()
        else 'defaults'
    ),
)

## MySQL用ボリューム

MySQLのデータを格納するボリュームに関するパラメータを指定します。

### MySQL用ボリュームのサイズを確認する

MySQL用ボリュームのサイズ(GB)を指定します。


> ハンズオン環境では事前に設定しているので設定値の確認のみ行います。

In [28]:
# (例)
# volume_database_size = 12

print(load_group_var(target_group, 'volume_database_size'))

8


### MySQL用ボリュームのボリュームタイプを確認する

ボリュームタイプを指定します。

> ハンズオン環境では事前に設定しているので設定値の確認のみ行います。

指定できるタイプはクラウドプロバイダ毎に異なります。どのような値が指定できるかについては以下のリンクを参考にしてください。

* AWS
  - [Amazon EBS ボリュームの種類](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/EBSVolumeTypes.html)
* Azure
  - [Managed Disks の価格](https://azure.microsoft.com/ja-jp/pricing/details/managed-disks/)

In [29]:
# (例)
# AWSの場合
# volume_database_type = 'gp2'
# Azureの場合
# volume_database_type = 'Standard_LRS'

print(load_group_var(target_group, 'volume_database_type'))

gp2


### MySQL用ボリュームのマウントポイントを指定する

MySQL用ボリュームをマウントするディレクトリを指定してください。

In [30]:
volume_database_mount = '/opt/mysql'

#### 保存

In [31]:
%run scripts/group.py
update_group_vars(
    target_group,
    volume_database_mount=volume_database_mount,
)

### MySQL用ボリュームのファイルシステムを指定する


MySQL用ボリュームのファイルシステムを指定してください。

今のところ指定できる値は **btrfs のみ**になっています。

In [32]:
volume_database_fs = 'btrfs'

ファイルシステムをマウントする際のオプションを指定してください。ここで値が指定されない場合はデフォルトの設定でマウントされます。

In [33]:
# (例)
# volume_database_mount_opt = 'noatime,autodefrag,compress-force=lzo,space_cache'

#### 保存

In [34]:
# サポートしているファイルシステムが指定されているかをチェックする
support_fs = ['btrfs']
if volume_database_fs not in support_fs:
    raise RuntimeError(
        "ERROR: Unsupported file system: {}".format(volume_database_fs))

# 保存
update_group_vars(
    target_group,
    volume_database_fs=volume_database_fs,
)

update_group_vars(
    target_group,
    volume_database_mount_opt=(
        volume_database_mount_opt
        if 'volume_database_mount_opt' in vars()
        else 'defaults'
    ),
)

## 保存したパラメータの確認

ここまで group_vars ファイルに保存したパラメータの内容を確認します。

In [35]:
!cat group_vars/{target_group}.yml

ansible_ssh_extra_args: -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null
ansible_ssh_private_key_file: /home/bit_kun/.ssh/id_rsa_moodle
ansible_ssh_public_key_file: /home/bit_kun/.ssh/id_rsa_moodle.pub
ansible_user: ansible
target_group: Moodle
vcp_flavor: small
vcp_provider: aws
volume_database_fs: btrfs
volume_database_mount: /opt/mysql
volume_database_mount_opt: defaults
volume_moodle_fs: btrfs
volume_moodle_mount: /opt/moodle
volume_moodle_mount_opt: defaults


# Moodleデータの暗号化

このNotebookで構築するMoodle環境では[LUKS](https://gitlab.com/cryptsetup/cryptsetup)を利用してMoodleデータの暗号化を行うことができます。ここでは暗号化に関するパラメータを設定します。

![Moodleデータの暗号化](image/moodle-1-07.png)

## 暗号化利用の有無を指定する

MySQL用ボリューム、Moodle用ボリュームの暗号化を有効にするかどうかを設定します。

In [36]:
# (例)
# volume_encrypt = True    # 暗号化を利用する場合
# volume_encrypt = False   # 暗号化を利用しない場合

volume_encrypt = True

### 保存


In [37]:
%run scripts/group.py
update_group_vars(
    target_group,
    volume_encrypt=volume_encrypt,
)

## 鍵ファイルを生成する

暗号化に使用する鍵ファイルを作成します。

> 暗号化を有効にする設定でない場合は、次のセルを実行しても何も処理されません。

In [38]:
if load_group_var(target_group, 'volume_encrypt'):
    import tempfile

    # 鍵ファイルの長さ(Vaultサーバの制限のため鍵ファイルサイズの上限は 256KBとなっている)
    key_length = 256 * 1024

    # /dev/urandom から鍵ファイルを作成
    (dh, keyfile_name) = tempfile.mkstemp()
    !dd if=/dev/urandom of={keyfile_name} count={key_length // 512}

    !ls -l {keyfile_name}

512+0 records in
512+0 records out
262144 bytes (262 kB) copied, 0.0224406 s, 11.7 MB/s
-rw------- 1 bit_kun users 262144 Jun 21 16:00 /tmp/tmpx8ek61s0


## 鍵ファイルの格納場所を指定する

生成した鍵ファイルを格納する Vault サーバ内のパスを指定してください。

In [39]:
# (例)
# vault_luks_key_path = 'cubbyhole/luks'

vault_luks_key_path = 'cubbyhole/luks'

### チェックと保存

鍵ファイルをVaultサーバに格納します。また格納場所のパスを group_vars ファイルに保存します。

In [40]:
# 暗号化が有効な場合のみ、チェックと保存を行う
if load_group_var(target_group, 'volume_encrypt'):
    import json
    import base64
    %run scripts/group.py
    
    vault_address = load_group_var(target_group, 'vault_address')
    vault_token = load_group_var(target_group, 'vcc_access_token')
    
    # 鍵ファイルを読み込む
    with open(keyfile_name, 'rb') as f:
        luks_key = f.read()
    
    # Vault サーバに書き込むために BASE64 でエンコードする
    luks_params = dict(
        keyfile=base64.b64encode(luks_key).decode('UTF-8'),
    )

    # curl に入力値を渡すための一時ファイルを作成する
    (th, tname) = tempfile.mkstemp()
    with open(th, 'w') as f:
        f.write(json.dumps(luks_params))
        
    try:
        # Vaultサーバに秘密情報を格納する
        !curl --fail --header "X-Vault-Token:{vault_token}" \
            --header "Content-Type: application/json" -X POST \
            --data "@{tname}" \
            "{vault_address}/v1/{vault_luks_key_path}"

        # group_vars への保存
        update_group_vars(
            target_group,
            vault_luks_key_path=vault_luks_key_path,
        )
        
    finally:
        # 不要になった一時ファイルを削除する
        os.unlink(keyfile_name)
        os.unlink(tname)

## 保存したパラメータの確認

ここまで group_vars ファイルに保存したパラメータの内容を確認します。

In [41]:
!cat group_vars/{target_group}.yml

ansible_ssh_extra_args: -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null
ansible_ssh_private_key_file: /home/bit_kun/.ssh/id_rsa_moodle
ansible_ssh_public_key_file: /home/bit_kun/.ssh/id_rsa_moodle.pub
ansible_user: ansible
target_group: Moodle
vault_luks_key_path: cubbyhole/luks
vcp_flavor: small
vcp_provider: aws
volume_database_fs: btrfs
volume_database_mount: /opt/mysql
volume_database_mount_opt: defaults
volume_encrypt: true
volume_moodle_fs: btrfs
volume_moodle_mount: /opt/moodle
volume_moodle_mount_opt: defaults


# VCノードのディレクトリ構成

![VCノードのディレクトリ構成](image/moodle-1-08.png)

Moodle, MySQLなどのアプリケーションコンテナの実行環境となるVCノードのディレクトリ構成についての設定を行います。

## Moodleに関するディレクトリを指定する


### Moodleのデータディレクトリを指定する

Moodleのアップロードファイル、一時データ、セッションデータ等を保存するディレクトリを指定してください。

> ここで指定するディレクトリは VC ノードにおけるディレクトリであり、Moodleのアプリケーションコンテナにおけるディレクトリではありません。そのためMoodleの `moodledata`として設定するディレクトリとは異なりますので注意してください。

In [42]:
import os
moodle_dir_contents = os.path.join(volume_moodle_mount, 'data')

print(moodle_dir_contents)

/opt/moodle/data


#### 保存

In [43]:
import os
%run scripts/group.py

# 絶対パスが指定されていることをチェックする
if not os.path.isabs(moodle_dir_contents):
    raise RuntimeError(
        "ERROR: should be absolute path: {}".format(moodle_dir_contents))

# 保存
update_group_vars(
    target_group,
    moodle_dir_contents=moodle_dir_contents,
)

### PHPファイルを格納するディレクトリを指定する

MoodleのPHPファイルを格納するディレクトリを指定してください。

In [44]:
import os
moodle_dir_scripts = os.path.join(volume_moodle_mount, 'php')

print(moodle_dir_scripts)

/opt/moodle/php


#### 保存

In [45]:
import os
%run scripts/group.py

# 絶対パスが指定されていることをチェックする
if not os.path.isabs(moodle_dir_scripts):
    raise RuntimeError(
        "ERROR: should be absolute path: {}".format(moodle_dir_scripts))

# 保存
update_group_vars(
    target_group,
    moodle_dir_scripts=moodle_dir_scripts,
)

### Moodleのログファイルを格納するディレクトリを指定する

Moodleの httpd サーバのログファイルを格納するディレクトリを指定してください。

In [46]:
moodle_dir_logs = '/var/log/moodle'

#### 保存

In [47]:
import os
%run scripts/group.py

# 絶対パスが指定されていることをチェックする
if not os.path.isabs(moodle_dir_logs):
    raise RuntimeError(
        "ERROR: should be absolute path: {}".format(moodle_dir_logs))

# 保存
update_group_vars(
    target_group,
    moodle_dir_logs=moodle_dir_logs,
)

## MySQLに関するディレクトリを指定する


### データベースに関するファイルを格納するディレクトリを指定する

MySQLのデータベースに関するファイルを格納するディレクトリを指定してください。

In [48]:
mysql_dir_data = volume_database_mount

#### 保存

In [49]:
import os
%run scripts/group.py

# 絶対パスが指定されていることをチェックする
if not os.path.isabs(mysql_dir_data):
    raise RuntimeError(
        "ERROR: should be absolute path: {}".format(mysql_dir_data))

# 保存
update_group_vars(
    target_group,
    mysql_dir_data=mysql_dir_data,
)

### MySQLのログファイルを格納するディレクトリを指定する

MySQLのログファイルを格納するディレクトリを指定してください。

In [50]:
mysql_dir_logs = '/var/log/mysql'

#### 保存

In [51]:
import os
%run scripts/group.py

# 絶対パスが指定されていることをチェックする
if not os.path.isabs(mysql_dir_logs):
    raise RuntimeError(
        "ERROR: should be absolute path: {}".format(mysql_dir_logs))

# 保存
update_group_vars(
    target_group,
    mysql_dir_logs=mysql_dir_logs,
)

## Nginxのログファイルを格納するディレクトリを指定する

Nginxのログファイルを格納するディレクトリを指定してください。

In [52]:
proxy_dir_logs = '/var/log/nginx'

### 保存

In [53]:
import os
%run scripts/group.py

# 絶対パスが指定されていることをチェックする
if not os.path.isabs(proxy_dir_logs):
    raise RuntimeError(
        "ERROR: should be absolute path: {}".format(proxy_dir_logs))

# 保存
update_group_vars(
    target_group,
    proxy_dir_logs=proxy_dir_logs,
)

## 構成ファイルなどを格納するディレクトリを指定する

アプリケーションコンテナの構成や設定ファイルなどを格納するディレクトリを指定してください。

In [54]:
project_dir = '/home/{}/moodle'.format(ansible_user)

### 保存

group_vars に保存する。

In [55]:
update_group_vars(
    target_group,
    project_dir=project_dir,
)

## 保存したパラメータの確認

ここまで group_vars ファイルに保存したパラメータの内容を確認します。

In [56]:
!cat group_vars/{target_group}.yml

ansible_ssh_extra_args: -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null
ansible_ssh_private_key_file: /home/bit_kun/.ssh/id_rsa_moodle
ansible_ssh_public_key_file: /home/bit_kun/.ssh/id_rsa_moodle.pub
ansible_user: ansible
moodle_dir_contents: /opt/moodle/data
moodle_dir_logs: /var/log/moodle
moodle_dir_scripts: /opt/moodle/php
mysql_dir_data: /opt/mysql
mysql_dir_logs: /var/log/mysql
project_dir: /home/ansible/moodle
proxy_dir_logs: /var/log/nginx
target_group: Moodle
vault_luks_key_path: cubbyhole/luks
vcp_flavor: small
vcp_provider: aws
volume_database_fs: btrfs
volume_database_mount: /opt/mysql
volume_database_mount_opt: defaults
volume_encrypt: true
volume_moodle_fs: btrfs
volume_moodle_mount: /opt/moodle
volume_moodle_mount_opt: defaults


# チェックと保存

設定項目漏れがないことを確認します。また固定値となるパラメータの設定も同時に行います。

## 設定項目漏れがないことをチェックする

必須項目が設定されているかをチェックします。

In [57]:
%run scripts/group.py
gvars = load_group_vars(target_group)
require_params = [
    'ansible_user', 'ansible_ssh_extra_args', 'target_group',
    'ansible_ssh_private_key_file', 'ansible_ssh_public_key_file',
    'vault_address',
    'vcp_provider', 'vcp_flavor', 'vcc_access_token', 'volume_encrypt',
    'volume_database_type', 'volume_database_size', 'volume_database_fs',
    'volume_database_mount', 'volume_database_mount_opt',
    'volume_moodle_type', 'volume_moodle_size', 'volume_moodle_fs',
    'volume_moodle_mount', 'volume_moodle_mount_opt',
    'moodle_dir_contents', 'moodle_dir_scripts', 'moodle_dir_logs',
    'mysql_dir_data', 'mysql_dir_logs', 'proxy_dir_logs', 'project_dir',
    'vc_node_address',
]
if gvars['vcp_provider'] == 'aws':
    require_params.append('aws_availability_zone')
    require_params.append('aws_region')
elif gvars['vcp_provider'] == 'azure':
    require_params.append('azure_resource_group_name')
    
if gvars['volume_encrypt']:
    require_params.append('vault_luks_key_path')

for x in require_params:
    if x not in gvars:
        raise RuntimeError("ERROR: not set {}".format(x))

## デフォルト設定を保存する

固定値となるパラメータについて group_vars ファイルに設定値を書き込みます。

このNotebookではMoodleのアップデートを行う手順などを提供しています。その際、アップデート前後のアプリケーションコンテナを区別するためのパラメータとして「コンテナバージョン」を用いています。ここではコンテナバージョンの初期値を設定します。

In [58]:
if 'container_version' not in gvars:
    update_group_vars(
        target_group,
        container_version=1,
    )  

役割毎のAnsibleのグループ名を設定します。１ノード構成の場合は全て target_group と同じになります。

In [59]:
update_group_vars(
    target_group,
    target_group_moodle=target_group,
    target_group_database=target_group,
    target_group_rproxy=target_group,
)

VCノードに設定するタグが指定されていない場合は空のタグ設定を行います。

In [60]:
gvars = load_group_vars(target_group)
if 'vcp_tags' not in gvars:
    update_group_vars(
        target_group,
        vcp_tags={},
    )

Moodle用ボリューム、MySQL用ボリュームに作成するファイルシステムにつけるラベルを設定します。

In [61]:
update_group_vars(
    target_group,
    volume_moodle_fs_label='moodle',
    volume_database_fs_label='mysql',
)

その他、この構成で固定値となるパラメータを group_vars ファイルに書き込みます。

In [62]:
update_group_vars(
    target_group,
    moodle_shibboleth_enable=False,
    proxy_tls_enable=True,
    project_template_name='moodle-https',
    proxy_config_name='nginx.conf',
    proxy_config_update_name='nginx-update.conf',
    proxy_config_dir='',
)

## 自動的に割り当てられるパラメータを設定する

設定されている他のパラメータや自動的に導き出されるパラメータ値を設定します。

アプリケーションコンテナの名前を設定します。この値は`container_version`から定まる値となります。

In [63]:
gvars = load_group_vars(target_group)
docker_container_name_mysql = 'mysql-{}'.format(
    gvars['container_version'])
docker_container_name_moodle = 'moodle-{}'.format(
    gvars['container_version'])
update_group_vars(
    target_group,
    docker_container_name_mysql=docker_container_name_mysql,
    docker_container_name_moodle=docker_container_name_moodle,
)

各アプリケーションコンテナのコンテナバージョン毎のディレクトリを設定します。

In [64]:
gvars = load_group_vars(target_group)
subvolume_name_mysql = '{}/db-{}'.format(
    gvars['mysql_dir_data'], gvars['container_version'])
subvolume_name_moodle_contents = '{}/contents-{}'.format(
    gvars['moodle_dir_contents'], gvars['container_version'])
subvolume_name_moodle_scripts = '{}/moodle-{}'.format(
    gvars['moodle_dir_scripts'], gvars['container_version'])
update_group_vars(
    target_group,
    subvolume_name_mysql=subvolume_name_mysql,
    subvolume_name_moodle_contents=subvolume_name_moodle_contents,
    subvolume_name_moodle_scripts=subvolume_name_moodle_scripts,
)

Moodle用ボリューム、MySQL用ボリュームに対応するデバイス名を設定します。デバイス名はクラウドプロバイダの設定値に応じて定まる値となります。

In [65]:
%run scripts/group.py
vcp_provider = load_group_var(target_group, 'vcp_provider')
if vcp_provider == 'aws':
    volume_database_device = '/dev/xvdf'
    volume_moodle_device = '/dev/xvdg'
elif vcp_provider == 'azure':
    volume_database_device = '/dev/sdc'
    volume_moodle_device = '/dev/sdd'
else:
    # 他のプロバイダは未対応
    # 必要に応じて適切な値を設定すること
    pass

volume_encrypt = load_group_var(target_group, 'volume_encrypt')
if volume_encrypt:
    # ボリュームを暗号化する場合は /dev/mapper/* に置き換える
    volume_database_device0 = volume_database_device
    volume_moodle_device0 = volume_moodle_device
    volume_database_device = '/dev/mapper/mysql'
    volume_moodle_device = '/dev/mapper/moodle'

update_group_vars(
    target_group,
    volume_database_device=volume_database_device,
    volume_moodle_device=volume_moodle_device,
)

if 'volume_database_device0' in vars():
    update_group_vars(
        target_group,
        volume_database_device0=volume_database_device0,
        volume_moodle_device0=volume_moodle_device0,
    )    